# API calls to weather data

In [2]:
# Import the necessary modules
import requests
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [3]:
# urls
url_forecast = "https://api.openweathermap.org/data/2.5/forecast?q=Amsterdam&appid=a60541e63440b4d6b13419f767d8ad01"
url_weather = "https://api.openweathermap.org/data/2.5/weather?q=Amsterdam&appid=8c5591da6ae1e144b26df53a17c7c506"

## Weather forecast api

In [33]:
# Call the weather forecast api
call_forecast = requests.get(url_forecast)
json = call_forecast.json()
items = {k: v for k, v in json.items()}

# Extract the relevant data from the API results. cod, message and cnt were not relevant so those columns will be left out
list_weather = items['list']
city = items['city']

# Create empty dictionary and dataframe. During the looping, they get filled in.
final_dict = {}
final_df = pd.DataFrame()

## In order to transform the data into the structure we desire, some looping need to be done.
# The ultimate goal is to create a dictionary with an array for each item.

# Loop through the list weather data
for i in list_weather:
    # Loop through the dictionary of each iteration
    for k, v in i.items():
        # Check if the type of the item is a dictionary
        if type(v) is dict:
            # Looping if the item is a dictionary
            for key, value in v.items():
                final_key = k + "." + str(key)
                if final_key not in final_dict:
                    final_dict[final_key] = [value]
                elif final_key in final_dict:
                    final_dict[final_key].append(value)
        # Checks if the item is a list
        elif type(v) is list:
            for key, value in v[0].items():
                final_key = k + "." + str(key)
                if final_key not in final_dict:
                    final_dict[final_key] = [value]
                elif final_key in final_dict:
                    final_dict[final_key].append(value)
        # Else, threat the item like a int or a str
        else:
            if k not in final_dict:
                final_dict[k] = [v]
            elif k in final_dict:
                final_dict[k].append(v)

# Loop through the final dictionary. Add each list of items in a separate columns
for k, v in final_dict.items():
    final_df[k] = pd.Series(v)

# Set the datetime value as the index
final_df.set_index('dt_txt', inplace = True)

# Create an empty dictionary 
cities = {}

# Loop through the city dictionary and apply the same methods as before
for k, v in city.items():
    # One item has a dictionary of itself
    if type(v) is dict:
        for i in v:
            key = 'city.' + k + '.' + str(i)
            cities[key] = v[i]
    else:
        cities['city.' + k] = v

# Create a dictionary where the items of the cities dict created earlier is multipied by the amount of rows of the dataframe
cities = {k: list(np.full(shape = len(final_df), fill_value = v)) for k, v in cities.items()}

# Loop trough the items and add it as a column in the dataframe
for k, v in cities.items():
    final_df[k] = v

# Print the head of the final dataframe
print(final_df.head())

# Create the file based on t
datetime = str(dt.now().strftime('%d-%m-%Y-%H-%M-%S'))
fname = 'weather-forecast-' + datetime + '.csv'
final_df.to_csv(fname) 

                             dt  main.temp  main.feels_like  main.temp_min  \
dt_txt                                                                       
2022-01-31 21:00:00  1643662800     277.92           273.69         277.39   
2022-02-01 00:00:00  1643673600     277.88           273.98         277.80   
2022-02-01 03:00:00  1643684400     277.94           274.02         277.94   
2022-02-01 06:00:00  1643695200     278.66           274.67         278.66   
2022-02-01 09:00:00  1643706000     279.70           275.50         279.70   

                     main.temp_max  main.pressure  main.sea_level  \
dt_txt                                                              
2022-01-31 21:00:00         277.92           1017            1017   
2022-02-01 00:00:00         277.88           1018            1018   
2022-02-01 03:00:00         277.95           1018            1018   
2022-02-01 06:00:00         278.66           1017            1017   
2022-02-01 09:00:00         279.70     

## Current weather api

In [32]:
# Call the weather api
weather = requests.get(url_weather)

# Set the results in a json format
json = weather.json()

# Create an empty dictionary that gets filled in later on
final_dict = {}

# Loop through the weather data items
for k, v in json.items(): 
    # Check if the item is a list
    if type(v) is dict:
        for i in v:
            key = k + '.' + str(i)
            final_dict[key] = v[i]
    elif type(v) is list:
        for i in v[0]:
            key = k + '.' + str(i)
            final_dict[key] = v[0][i]  
    else:
        final_dict[k] = v
        
df = pd.DataFrame(final_dict, index = [final_dict['dt']])

# Print the head of the final dataframe
print(df.head())

# Create the filename
datetime = str(dt.now().strftime('%d-%m-%Y-%H-%M-%S'))
fname = 'current-weather-' + datetime + '.csv'
df.to_csv(fname, index_label = 'dt') 

            coord.lon  coord.lat  weather.id weather.main weather.description  \
1643659496     4.8897     52.374         802       Clouds    scattered clouds   

           weather.icon      base  main.temp  main.feels_like  main.temp_min  \
1643659496          03n  stations     278.12           274.96         276.93   

            ...          dt  sys.type   sys.id  sys.country  sys.sunrise  \
1643659496  ...  1643659496         2  2005807           NL   1643613812   

            sys.sunset  timezone       id       name  cod  
1643659496  1643646243      3600  2759794  Amsterdam  200  

[1 rows x 28 columns]
